In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# PERIODO

In [2]:
start_date = '2022-01-01'
final_date = '2022-01-31'

final_date_1 = '2022-01-15'
start_date_1 = '2022-01-16'

# BIN AMEX

In [3]:
bin_amex = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\BIN AMEX.txt',dtype={'BIN_RANGE':object,'COUNTRY_CODE':object})
bin_amex.fillna('null',inplace=True)
bin_amex.loc[bin_amex['COUNTRY_CODE'].str.contains('484'), "origen_tarjeta"] = 'NACIONAL'
bin_amex['origen_tarjeta'].fillna('INTERNACIONAL',inplace=True)
bin_amex['bin_bank'] = (bin_amex['BIN_RANGE'].astype(str).replace(' ', '',regex=True))
bin_amex.head()

,BIN_RANGE,PRODUCT_TYPE,COUNTRY_CODE,COUNTRY_NAME,CPS_OPTION,20211212,origen_tarjeta,bin_bank
0,370395,DEBIT,840,UNITED STATES OF AMERICA,N,null,INTERNACIONAL,370395
1,379983,CONSUMER,156,CHINA,N,null,INTERNACIONAL,379983
2,378256,CONSUMER,156,CHINA,N,null,INTERNACIONAL,378256
3,370524,CONSUMER,156,CHINA,N,null,INTERNACIONAL,370524
4,377659,CONSUMER,156,CHINA,N,null,INTERNACIONAL,377659


In [4]:
bin_amex.shape

(17501, 8)

# PARAMETRIA COMISIONES

In [5]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\PARAMETRIA AMEX.csv',dtype={'meses_msi':object,
                                                                                              'comision_porcentaje':float,
                                                                                              'industria_afiliacion':object})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
#parametria_comisiones['fin'] = pd.to_datetime(parametria_comisiones['fin'])
parametria_comisiones['comision_porcentaje_mdr'] = -0.0240
parametria_comisiones['comision_porcentaje_msi'] = -parametria_comisiones['comision_porcentaje']/100
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('EPA'), "affiliation_type"] = 'AGGREGATOR'
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('GRRCN'), "affiliation_type"] = 'OPTBLUE'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('99'),'meses_msi'] = '0'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('0'),'comision_porcentaje_msi'] = 0
parametria_comisiones['comision_porcentaje_final'] = (parametria_comisiones['comision_porcentaje_mdr'] +
                                                      parametria_comisiones['comision_porcentaje_msi'])
parametria_comisiones['index_comisiones'] = (parametria_comisiones['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'+
                                             parametria_comisiones['meses_msi'].astype(str).replace(' ', '',regex=True))
parametria_comisiones['index_comisiones_industria'] = (parametria_comisiones['industria_afiliacion'].astype(str).replace(' ', '',regex=True)+'_'+
                                                    (parametria_comisiones['affiliation_type'].astype(str).replace(' ', '',regex=True))+'_'+
                                                    (parametria_comisiones['meses_msi'].astype(str).replace(' ', '',regex=True)))
parametria_comisiones.loc[~(parametria_comisiones['index_comisiones_industria'].str.contains('TODAS')), "index_comisiones"] = 'NO_CONSIDERAR'
parametria_comisiones.head()

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,comision_porcentaje,comision_monto,proceso,comentario,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones,index_comisiones_industria
0,Null,Fija,VIGENTE,2018-01-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,EPA,...,2.40,0.00,Agregador,Costo de txn Amex Agregador General,-0.02,0.00,AGGREGATOR,-0.02,NO_CONSIDERAR,9351789434_AGGREGATOR_0
1,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,2.50,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,-0.02,-0.03,AGGREGATOR,-0.05,NO_CONSIDERAR,9351789434_AGGREGATOR_3
2,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.10,0.00,Agregador,Costo por el uso de MSI adicional a la tasa y ...,-0.02,-0.00,AGGREGATOR,-0.03,NO_CONSIDERAR,9351789434_AGGREGATOR_3
3,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,4.50,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,-0.02,-0.04,AGGREGATOR,-0.07,NO_CONSIDERAR,9351789434_AGGREGATOR_6
4,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.10,0.00,Agregador,Costo por el uso de MSI adicional a la tasa y ...,-0.02,-0.00,AGGREGATOR,-0.03,NO_CONSIDERAR,9351789434_AGGREGATOR_6


In [6]:
parametria_comisiones.shape

(55, 28)

In [7]:
parametria_comisiones['fuente'].unique()

array(['EPA', 'GRRCN'], dtype=object)

In [8]:
parametria_comisiones['meses_msi'].unique()

array(['0', '3', '6', '9', '12', '18', '15'], dtype=object)

# COMISIONES GRRCN COBRADAS

In [9]:
comisiones_grrcn = parametria_comisiones[(parametria_comisiones['fuente'] == 'GRRCN') &
                                       (parametria_comisiones['status'].str.contains('VIGENTE')) &
                                       (~parametria_comisiones['fin'].str.contains('2021-04-15'))]
comisiones_grrcn.head()

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,comision_porcentaje,comision_monto,proceso,comentario,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones,index_comisiones_industria
15,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,0.20,0.00,OptBlue,NF Network Fee Uso de Red,-0.02,0.00,OPTBLUE,-0.02,OPTBLUE_0,TODAS_OPTBLUE_0
16,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,0.40,0.00,OptBlue,IB Inbound fee Uso de tarjeta internacional,-0.02,0.00,OPTBLUE,-0.02,OPTBLUE_0,TODAS_OPTBLUE_0
17,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,0.65,0.00,OptBlue,1A MDR Costo de tasa industria EDUCATION,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464193_OPTBLUE_0
18,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,1.65,0.00,OptBlue,1A MDR Costo de tasa industria AUTO RENTAL,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464219_OPTBLUE_0
19,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,1.50,0.00,OptBlue,1A MDR Costo de tasa industria RETAIL,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464227_OPTBLUE_0


In [10]:
comisiones_grrcn.shape

(34, 28)

In [11]:
comisiones_grrcn['index_comisiones'].unique()

array(['OPTBLUE_0', 'NO_CONSIDERAR', 'OPTBLUE_3', 'OPTBLUE_6',
       'OPTBLUE_9', 'OPTBLUE_12', 'OPTBLUE_15', 'OPTBLUE_18'],
      dtype=object)

In [12]:
searchfor_card_present = ['8138464193',
'8138464219',
'8138464227',
'8138464185',
'8138464177',
'8138464151',
'8138464144',
'8138464136',
'8138464128',
'8138464441',
'8138464110']
searchfor_card_not_present = ['8482702172',
'8482702149',
'8482707999',
'8482702206',
'8482702263',
'8482702271',
'8482702297',
'8482707973',
'8482708005',
'8482702248',
'8482708013',
'8482702164']

In [13]:
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_present)), "industria_fee"] = comisiones_grrcn['comision_porcentaje']/100
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_not_present)), "industria_fee"] = comisiones_grrcn['comision_porcentaje']/100
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_present)), "tipo_tarjeta"] = 'card_present'
comisiones_grrcn.loc[(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')) &
                     comisiones_grrcn['industria_afiliacion'].str.contains('|'.join(searchfor_card_not_present)), "tipo_tarjeta"] = 'card_not_present'
comisiones_grrcn.loc[(comisiones_grrcn['fuente'] == 'GRRCN'), "network_fee"] = 0.2/100
comisiones_grrcn.loc[~(comisiones_grrcn['index_comisiones_industria'].str.contains('OPTBLUE_0')), "sobretasa"] = comisiones_grrcn['comision_porcentaje']/100
comisiones_grrcn.loc[~(comisiones_grrcn['comision_aplicada'].str.contains('SOBRE TASA')), "index_comisiones"] = 'NO_CONSIDERAR'
comisiones_grrcn['index_comisiones_industria'] = comisiones_grrcn['index_comisiones_industria'].str[:-2]
comisiones_grrcn['industria_fee'].fillna(0,inplace=True)
comisiones_grrcn['sobretasa'].fillna(0,inplace=True)
comisiones_grrcn.head()


C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones,index_comisiones_industria,industria_fee,tipo_tarjeta,network_fee,sobretasa
15,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,TODAS_OPTBLUE,0.00,NaN,0.00,0.00
16,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,TODAS_OPTBLUE,0.00,NaN,0.00,0.00
17,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464193_OPTBLUE,0.01,card_present,0.00,0.00
18,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464219_OPTBLUE,0.02,card_present,0.00,0.00
19,Null,Fija,VIGENTE,2021-02-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,GRRCN,...,-0.02,0.00,OPTBLUE,-0.02,NO_CONSIDERAR,8138464227_OPTBLUE,0.01,card_present,0.00,0.00


In [14]:
comisiones_grrcn.shape

(34, 32)

In [15]:
comisiones_grrcn['industria_fee'].unique()

array([0.    , 0.0065, 0.0165, 0.015 , 0.013 , 0.018 , 0.0105, 0.0185,
       0.0195, 0.004 , 0.016 , 0.019 ])

In [16]:
comisiones_grrcn['index_comisiones'].unique()

array(['NO_CONSIDERAR', 'OPTBLUE_3', 'OPTBLUE_6', 'OPTBLUE_9',
       'OPTBLUE_12', 'OPTBLUE_15', 'OPTBLUE_18'], dtype=object)

# DB

In [17]:
# db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX SEPTIEMBRE 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX AGOSTO 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
# db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX OCTUBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_3 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX JUNIO-JULIO 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_4 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2022\AMEX ENERO 2022.csv'
                   ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
                            ,'MCC':object})

In [18]:
db_first = pd.concat([db_1])
db_first.columns = db_first.columns.str.strip().str.lower() 
db_first['order_id'] = db_first['order_id'].str.strip()
db_first['order_id'] = db_first['order_id'].str.upper()
db_first['date_transaction'] = pd.to_datetime(db_first['date_transaction'])
db_first.loc[~db_first['status_code'].str.contains('3'),'amount_new'] = db_first['amount']
db_first.loc[db_first['status_code'].str.contains('3'),'amount_new'] = db_first['amount']*-1
db_first['INDEX_TOTAL'] = (db_first['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_first['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_first['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_first['amount'].astype(str).replace(' ', '',regex=True))
db_first['INDEX_PARCIAL'] = (db_first['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_first['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_first['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                       +db_first['amount'].astype(str).replace(' ', '',regex=True))
db_first['order_id'].fillna('SIN ORDER_ID EN DB',inplace=True)
db_first['index_comisiones'] = ('OPTBLUE'+'_'+db_first['msi'].astype(str).str[:-6].replace(' ', '',regex=True))
db_first['index_comisiones_industria'] = (db_first['affiliation']+'_'+'OPTBLUE')
db_first.rename(columns ={'bin':'bin_bank'}, inplace=True)
db_first.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,device_id,clip_reader,status_msg,issuer,brand,amount_new,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,A36886700809F09FFF0928C8A3,Clip Plus 2,Paid,AMEX,American Express,750.00,376654_3744_JSLUWR8TVTEQ_750.0,376654_3744_001453025292483_750.0,OPTBLUE_0,8138464128_OPTBLUE
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,P52108150002,Clip Classic,Paid,CIBC,AMERICAN EXPRESS,100.00,379180_3350_AAGQHWJVP62L_100.0,379180_3350_001453018851485_100.0,OPTBLUE_0,8138464227_OPTBLUE
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,PP36812120010207,Clip Pro,Paid,AMEX,American Express,135.00,371777_2015_HTQH36XWKT1T_135.0,371777_2015_001453023433485_135.0,OPTBLUE_0,8138464227_OPTBLUE
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,A368437204000760FF09244FF0,Clip Plus 2,Paid,AMEX,American Express,165.00,376718_2003_GZ1OVVUY5H5G_165.0,376718_2003_001453020571485_165.0,OPTBLUE_0,8138464136_OPTBLUE
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,37303033373630390A270008,Clip Plus,Paid,AMEX,American Express,300.00,376702_1007_MJYCGQD0QM1C_300.0,376702_1007_001453018868489_300.0,OPTBLUE_0,8138464128_OPTBLUE


In [19]:
db_first.shape

(170345, 29)

In [20]:
db_first['index_comisiones'].unique()

array(['OPTBLUE_0', 'OPTBLUE_9', 'OPTBLUE_12', 'OPTBLUE_3', 'OPTBLUE_6'],
      dtype=object)

### DB CON BIN

In [21]:
# db_first_bin = db_first.merge(bin_amex[['bin_bank','origen_tarjeta']],on='bin_bank',how='left')
# db_first_bin['origen_tarjeta'].fillna('INTERNACIONAL',inplace=True)
# db_first_bin.head()

In [22]:
# db_first_bin.shape

### DB CON COMISIONES

In [23]:
# db_first_comisiones_industria = db_first_bin.merge(comisiones_grrcn[['index_comisiones_industria','industria_fee',
#                                                                         'tipo_tarjeta']],
#                                                    on='index_comisiones_industria',how='left')
# db_first_comisiones_industria.head()

In [24]:
# db_first_comisiones_industria.shape

In [25]:
# db_second = db_first_comisiones_industria.merge(comisiones_grrcn[['index_comisiones','sobretasa']],
#                                         on='index_comisiones',how='left')

# db_second.loc[(db_second['origen_tarjeta'] == 'INTERNACIONAL'), "inbound_fee"] = 0.4/100

# db_second.loc[(db_second['affiliation_type'] == 'OPTBLUE'), "network_fee"] = 0.2/100

# db_second.loc[((db_second['amount'] >= 0) &
#                       (db_second['amount'] < 1000)), "sobrerango_v0"] = 0

# db_second.loc[((db_second['amount'] >= 1000) &
#                       (db_second['amount'] < 3000)), "sobrerango_v1"] = 0.4/100

# db_second.loc[((db_second['amount'] >= 3000) &
#                       (db_second['amount'] < 10000)), "sobrerango_v2"] = 0.7/100

# db_second.loc[((db_second['amount'] >= 10000) &
#                       (db_second['amount'] < 1000000)), "sobrerango_v3"] = 1.3/100

# db_second.loc[((db_second['amount'] < 0) &
#                       (db_second['amount'] >= -1000)), "sobrerango_v0"] = 0

# db_second.loc[((db_second['amount'] < -1000) &
#                       (db_second['amount'] >= -2999.99)), "sobrerango_v1"] = -0.4/100

# db_second.loc[((db_second['amount'] < -3000) &
#                       (db_second['amount'] >= -9999.99)), "sobrerango_v2"] = -0.7/100

# db_second.loc[((db_second['amount'] < -10000) &
#                       (db_second['amount'] >= -999999.99)), "sobrerango_v3"] = -1.3/100

# db_second['sobretasa'].fillna(0,inplace=True)
# db_second['inbound_fee'].fillna(0,inplace=True)
# db_second['sobrerango_v0'].fillna(0,inplace=True)
# db_second['sobrerango_v1'].fillna(0,inplace=True)
# db_second['sobrerango_v2'].fillna(0,inplace=True)
# db_second['sobrerango_v3'].fillna(0,inplace=True)

# db_second['monto_sobrerango_v0'] =  db_second['sobrerango_v0']*100
# db_second['iva_monto_sobrerango_v0'] =  db_second['monto_sobrerango_v0']*0.1600

# db_second['monto_sobrerango_v1'] =  db_second['sobrerango_v1']*100
# db_second['iva_monto_sobrerango_v1'] =  db_second['monto_sobrerango_v1']*0.1600

# db_second['monto_sobrerango_v2'] =  db_second['sobrerango_v2']*100
# db_second['iva_monto_sobrerango_v2'] =  db_second['monto_sobrerango_v2']*0.1600

# db_second['monto_sobrerango_v3'] =  db_second['sobrerango_v3']*100
# db_second['iva_monto_sobrerango_v3'] =  db_second['monto_sobrerango_v3']*0.1600

# db_second['monto_comision_industria_fee'] =  db_second['amount']*(db_second['industria_fee'])
# db_second['iva_monto_comision_industria_fee'] =  db_second['monto_comision_industria_fee']*0.1600

# db_second['monto_comision_inbound_fee'] = db_second['amount']*(db_second['inbound_fee'])
# db_second['iva_monto_comision_inbound_fee'] =  db_second['monto_comision_inbound_fee']*0.1600

# db_second['monto_comision_network_fee'] = db_second['amount']*(db_second['network_fee'])
# db_second['iva_monto_comision_network_fee'] =  db_second['monto_comision_network_fee']*0.1600

# db_second.loc[(db_second['monto_comision_network_fee'] < 0), 'monto_comision_cancelacion_network_fee'] = db_second['monto_comision_network_fee']*-1
# db_second['iva_monto_comision_cancelacion_network_fee'] =  db_second['monto_comision_cancelacion_network_fee']*0.1600

# db_second.loc[(db_second['monto_comision_inbound_fee'] < 0), 'monto_comision_cancelacion_inbound_fee'] = db_second['monto_comision_inbound_fee']*-1
# db_second['iva_monto_comision_cancelacion_inbound_fee'] =  db_second['monto_comision_cancelacion_inbound_fee']*0.1600

# db_second['monto_comision_cancelacion_network_fee'].fillna(0,inplace=True)
# db_second['iva_monto_comision_cancelacion_network_fee'].fillna(0,inplace=True)
# db_second['monto_comision_cancelacion_inbound_fee'].fillna(0,inplace=True)
# db_second['iva_monto_comision_cancelacion_inbound_fee'].fillna(0,inplace=True)

# db_second['monto_comision_cancelacion'] = (db_second['monto_comision_cancelacion_network_fee'] +
#                                                   db_second['monto_comision_cancelacion_inbound_fee'])
# db_second['iva_monto_comision_cancelacion'] = (db_second['iva_monto_comision_cancelacion_network_fee'] +
#                                                       db_second['iva_monto_comision_cancelacion_inbound_fee'])

# db_second['monto_comision_cancelacion'].fillna(0,inplace=True)
# db_second['iva_monto_comision_cancelacion'].fillna(0,inplace=True)

# db_second['monto_sobretasa'] = db_second['amount']*(db_second['sobretasa'])
# db_second['iva_monto_sobretasa'] =  db_second['monto_sobretasa']*0.1600

# db_second['total_monto_sobrerango'] =  (db_second['monto_sobrerango_v0'] +
#                                                db_second['monto_sobrerango_v1'] +
#                                                db_second['monto_sobrerango_v2'] +
#                                                db_second['monto_sobrerango_v3'])
# db_second['total_iva_monto_sobrerango'] =  (db_second['iva_monto_sobrerango_v0'] +
#                                                    db_second['iva_monto_sobrerango_v1'] +
#                                                    db_second['iva_monto_sobrerango_v2'] +
#                                                    db_second['iva_monto_sobrerango_v3'])

# db_second['total_monto_comisiones'] =  (db_second['monto_comision_industria_fee'] +
#                                                db_second['monto_comision_inbound_fee'] +
#                                                db_second['monto_comision_network_fee'] +
#                                                db_second['monto_comision_cancelacion'])
# db_second['total_iva_monto_comisiones'] =  (db_second['iva_monto_comision_industria_fee'] +
#                                                     db_second['iva_monto_comision_inbound_fee'] +
#                                                     db_second['iva_monto_comision_network_fee'] +
#                                                     db_second['iva_monto_comision_cancelacion'])

# db_second.head()

In [26]:
# db_second.shape

In [27]:
# duplicates_db = db_second[db_second.duplicated(['transaction_id'] , keep=False)]
# duplicates_db.head()

In [28]:
# db_second.shape

In [29]:
# db_second['affiliation_type'].unique()

### GRRCN ONLY

In [30]:
db = db_first[(db_first['affiliation_type'] == 'OPTBLUE')]
db.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,device_id,clip_reader,status_msg,issuer,brand,amount_new,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,A36886700809F09FFF0928C8A3,Clip Plus 2,Paid,AMEX,American Express,750.00,376654_3744_JSLUWR8TVTEQ_750.0,376654_3744_001453025292483_750.0,OPTBLUE_0,8138464128_OPTBLUE
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,P52108150002,Clip Classic,Paid,CIBC,AMERICAN EXPRESS,100.00,379180_3350_AAGQHWJVP62L_100.0,379180_3350_001453018851485_100.0,OPTBLUE_0,8138464227_OPTBLUE
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,PP36812120010207,Clip Pro,Paid,AMEX,American Express,135.00,371777_2015_HTQH36XWKT1T_135.0,371777_2015_001453023433485_135.0,OPTBLUE_0,8138464227_OPTBLUE
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,A368437204000760FF09244FF0,Clip Plus 2,Paid,AMEX,American Express,165.00,376718_2003_GZ1OVVUY5H5G_165.0,376718_2003_001453020571485_165.0,OPTBLUE_0,8138464136_OPTBLUE
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,37303033373630390A270008,Clip Plus,Paid,AMEX,American Express,300.00,376702_1007_MJYCGQD0QM1C_300.0,376702_1007_001453018868489_300.0,OPTBLUE_0,8138464128_OPTBLUE


In [31]:
db.shape

(169664, 29)

# GRRNC

In [32]:
allfiles = glob.glob(r'G:\My Drive\GRRCN\202*\**\OPTBLUE -*\*.txt', recursive=True)
allfiles

['G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-18.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-19.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-22.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-23.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-17.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-16.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-15.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-12.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-24.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE - PRODUCCION\\OPTBLUE GRRCN 2021-02-25.txt',
 'G:\\My Drive\\GRRCN\\2021\\02. FEBRERO\\OPTBLUE 

In [33]:
names_header=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'
              ,'22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']

In [34]:
grrcn_file = pd.concat([pd.read_csv(f,names=names_header, sep=',',dtype=object) for f in allfiles])
grrcn_file.head()

,1,2,3,4,5,6,7,8,9,10,...,32,33,34,35,36,37,38,39,40,41
0,HEADER,20210218,021942,0000000011,GRRCN,PAYCLIP MEXICO,2.01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SUMMARY,8138352885,002,0480082304,20210218,MXN,NaN,000000077567445,000000080375882,000000001368031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SUBMISSION,8138352885,002,0480082304,20210218,MXN,8138464110,20210217,20210217,000000000049009,...,000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRANSACTN,8138352885,002,0480082304,20210218,MXN,8138464110,20210217,20210217,000000000049009,...,NaN,,000000000190000,000000000190000,00012,00012,NaN,000000000000130,000000000000000,NaN
4,TXNPRICING,8138352885,002,0480082304,20210218,MXN,8138464110,NaN,NaN,9FJSZWNETOXH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
grrcn_file.shape

(8362737, 41)

In [36]:
grrcn_file['1'].unique()

array(['HEADER', 'SUMMARY', 'SUBMISSION', 'TRANSACTN', 'TXNPRICING',
       'ADJUSTMENT', 'TRAILER', 'CHARGEBACK'], dtype=object)

In [37]:
grrcn_head = grrcn_file[(grrcn_file['1'] == 'HEADER')]
grrcn_summ = grrcn_file[(grrcn_file['1'] == 'SUMMARY')]
grrcn_subm = grrcn_file[(grrcn_file['1'] == 'SUBMISSION')]
grrcn_tran = grrcn_file[(grrcn_file['1'] == 'TRANSACTN')]
grrcn_txnp = grrcn_file[(grrcn_file['1'] == 'TXNPRICING')]
grrcn_char = grrcn_file[(grrcn_file['1'] == 'CHARGEBACK')]
grrcn_adju = grrcn_file[(grrcn_file['1'] == 'ADJUSTMENT')]
grrcn_trai = grrcn_file[(grrcn_file['1'] == 'TRAILER')]

### TRANSACTN 

In [50]:
grrcn_tran['5'] = pd.to_datetime(grrcn_tran['5'], format='%Y%m%d').dt.date
grrcn_tran['8'] = pd.to_datetime(grrcn_tran['8'], format='%Y%m%d').dt.date
grrcn_tran['9'] = pd.to_datetime(grrcn_tran['9'], format='%Y%m%d').dt.date
grrcn_tran['19'] = pd.to_datetime(grrcn_tran['19'], format='%Y%m%d').dt.date
grrcn_tran['20'] = pd.to_datetime(grrcn_tran['20'], format='%H%M%S').dt.time
grrcn_tran['5'] = pd.to_datetime(grrcn_tran['5'])
grrcn_tran['8'] = pd.to_datetime(grrcn_tran['8'])
grrcn_tran['9'] = pd.to_datetime(grrcn_tran['9'])
grrcn_tran['19'] = pd.to_datetime(grrcn_tran['19'])
grrcn_tran.sort_values(by=['19','20'],inplace=True)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [51]:
grrcn_tran.rename(columns={'5':'fecha_pago','7':'numero_establecimiento','8':'fecha_presentacion','9':'fecha_procesamiento'
                           ,'15':'no_tarjeta','13':'order_id','22':'codigo_aprobacion','23':'numero_terminal'
                           ,'26':'referencia_adquiriente','36':'cantidad_cuotas','37':'numero_cuota'
                          ,'19':'fecha_transaccion','20':'hora_transaccion','21':'numero_transaccion',
                          '33':'rechazo_amex'},inplace=True)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [52]:
grrcn_tran['17'] = grrcn_tran['17'].astype(float)
grrcn_tran['18'] = grrcn_tran['18'].astype(float)
grrcn_tran['34'] = grrcn_tran['34'].astype(float)
grrcn_tran['35'] = grrcn_tran['35'].astype(float)
grrcn_tran['39'] = grrcn_tran['39'].astype(float)
grrcn_tran['40'] = grrcn_tran['40'].astype(float)
grrcn_tran['cantidad_cuotas'] = grrcn_tran['cantidad_cuotas'].astype(float)
grrcn_tran['numero_cuota'] = grrcn_tran['numero_cuota'].astype(float)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [53]:
grrcn_tran['Monto_bruto_presentacion'] = grrcn_tran['17']/100
grrcn_tran['Monto_transaccion'] = grrcn_tran['18']/100
grrcn_tran['Monto_primera_cuota'] = grrcn_tran['34']/100
grrcn_tran['Monto_cuota_siguiente'] = grrcn_tran['35']/100
grrcn_tran['Monto_servicio'] = grrcn_tran['39']/100
grrcn_tran['Monto_aceleracion'] = grrcn_tran['40']/100
grrcn_tran['last4_bank'] = grrcn_tran['no_tarjeta'].str[-4:].astype(str)
grrcn_tran['bin_bank'] = grrcn_tran['no_tarjeta'].str[:6].astype(str)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [54]:
grrcn_tran['INDEX_TOTAL'] = (grrcn_tran['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['Monto_transaccion'].astype(str).replace(' ', '',regex=True))
grrcn_tran['INDEX_PARCIAL'] = (grrcn_tran['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['codigo_aprobacion'].astype(str).replace(' ', '',regex=True)+'_'
                            +grrcn_tran['Monto_transaccion'].astype(str).replace(' ', '',regex=True))
grrcn_tran['index_comisiones'] = ('OPTBLUE'+'_'+grrcn_tran['numero_cuota'].astype(str).str[:-2].replace(' ', '',regex=True))
grrcn_tran['index_comisiones_industria'] = (grrcn_tran['numero_establecimiento']+'_'+'OPTBLUE')

grrcn_tran.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_primera_cuota,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria
121,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130336,...,0.00,0.00,0.00,0.00,2001,371775,371775_2001_nan_2.5,371775_2001_886305_2.5,OPTBLUE_0,8138464136_OPTBLUE
128,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130337,...,0.00,0.00,0.00,0.00,2001,371775,371775_2001_nan_3.0,371775_2001_811212_3.0,OPTBLUE_0,8138464136_OPTBLUE
142,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130338,...,360.00,360.00,0.00,0.00,1023,376702,376702_1023_nan_2160.0,376702_1023_874555_2160.0,OPTBLUE_6,8138464136_OPTBLUE
162,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130339,...,0.00,0.00,0.00,0.00,2003,371774,371774_2003_nan_195.0,371774_2003_842062_195.0,OPTBLUE_0,8138464136_OPTBLUE
169,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130340,...,0.00,0.00,0.00,0.00,5008,370786,370786_5008_nan_504.0,370786_5008_878155_504.0,OPTBLUE_0,8138464136_OPTBLUE


In [55]:
grrcn_tran.shape

(1148339, 53)

In [56]:
grrcn_tran['index_comisiones'].unique()

array(['OPTBLUE_0', 'OPTBLUE_6', 'OPTBLUE_12', 'OPTBLUE_3', 'OPTBLUE_9'],
      dtype=object)

### TRANSACTN CON BIN

In [57]:
grrcn_tran_bin = grrcn_tran.merge(bin_amex[['bin_bank','origen_tarjeta']],on='bin_bank',how='left')
grrcn_tran_bin['origen_tarjeta'].fillna('INTERNACIONAL',inplace=True)
grrcn_tran_bin.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_cuota_siguiente,Monto_servicio,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria,origen_tarjeta
0,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130336,...,0.00,0.00,0.00,2001,371775,371775_2001_nan_2.5,371775_2001_886305_2.5,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL
1,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130337,...,0.00,0.00,0.00,2001,371775,371775_2001_nan_3.0,371775_2001_811212_3.0,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL
2,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130338,...,360.00,0.00,0.00,1023,376702,376702_1023_nan_2160.0,376702_1023_874555_2160.0,OPTBLUE_6,8138464136_OPTBLUE,NACIONAL
3,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130339,...,0.00,0.00,0.00,2003,371774,371774_2003_nan_195.0,371774_2003_842062_195.0,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL
4,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130340,...,0.00,0.00,0.00,5008,370786,370786_5008_nan_504.0,370786_5008_878155_504.0,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL


In [58]:
grrcn_tran_bin.shape

(1148339, 54)

In [59]:
grrcn_tran_bin['origen_tarjeta'].unique()

array(['NACIONAL', 'INTERNACIONAL'], dtype=object)

### TRANSACTN CON COMISIONES

In [60]:
grrcn_tran_comisiones_industria = grrcn_tran_bin.merge(comisiones_grrcn[['index_comisiones_industria','industria_fee',
                                                                        'tipo_tarjeta']]
                                                       ,on='index_comisiones_industria',how='left')
grrcn_tran_comisiones_industria.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,Monto_aceleracion,last4_bank,bin_bank,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,index_comisiones_industria,origen_tarjeta,industria_fee,tipo_tarjeta
0,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130336,...,0.00,2001,371775,371775_2001_nan_2.5,371775_2001_886305_2.5,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL,0.02,card_present
1,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130337,...,0.00,2001,371775,371775_2001_nan_3.0,371775_2001_811212_3.0,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL,0.02,card_present
2,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130338,...,0.00,1023,376702,376702_1023_nan_2160.0,376702_1023_874555_2160.0,OPTBLUE_6,8138464136_OPTBLUE,NACIONAL,0.02,card_present
3,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130339,...,0.00,2003,371774,371774_2003_nan_195.0,371774_2003_842062_195.0,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL,0.02,card_present
4,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130340,...,0.00,5008,370786,370786_5008_nan_504.0,370786_5008_878155_504.0,OPTBLUE_0,8138464136_OPTBLUE,NACIONAL,0.02,card_present


In [61]:
grrcn_tran_comisiones_industria.shape

(1148339, 56)

In [62]:
grrcn_tran_final = grrcn_tran_comisiones_industria.merge(comisiones_grrcn[['index_comisiones','sobretasa']],
                                        on='index_comisiones',how='left')

grrcn_tran_final.loc[(grrcn_tran_final['origen_tarjeta'] == 'INTERNACIONAL'), "inbound_fee"] = 0.4/100

grrcn_tran_final.loc[(grrcn_tran_final['1'] == 'TRANSACTN'), "network_fee"] = 0.2/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 0) &
                      (grrcn_tran_final['Monto_transaccion'] < 1000)), "sobrerango_v0"] = 0

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 1000) &
                      (grrcn_tran_final['Monto_transaccion'] < 3000)), "sobrerango_v1"] = 0.4/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 3000) &
                      (grrcn_tran_final['Monto_transaccion'] < 10000)), "sobrerango_v2"] = 0.7/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] >= 10000) &
                      (grrcn_tran_final['Monto_transaccion'] < 1000000)), "sobrerango_v3"] = 1.3/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < 0) &
                      (grrcn_tran_final['Monto_transaccion'] >= -1000)), "sobrerango_v0"] = 0

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < -1000) &
                      (grrcn_tran_final['Monto_transaccion'] >= -2999.99)), "sobrerango_v1"] = -0.4/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < -3000) &
                      (grrcn_tran_final['Monto_transaccion'] >= -9999.99)), "sobrerango_v2"] = -0.7/100

grrcn_tran_final.loc[((grrcn_tran_final['Monto_transaccion'] < -10000) &
                      (grrcn_tran_final['Monto_transaccion'] >= -999999.99)), "sobrerango_v3"] = -1.3/100

grrcn_tran_final['sobretasa'].fillna(0,inplace=True)
grrcn_tran_final['inbound_fee'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v0'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v1'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v2'].fillna(0,inplace=True)
grrcn_tran_final['sobrerango_v3'].fillna(0,inplace=True)

grrcn_tran_final['monto_sobrerango_v0'] =  grrcn_tran_final['sobrerango_v0']*100
grrcn_tran_final['iva_monto_sobrerango_v0'] =  grrcn_tran_final['monto_sobrerango_v0']*0.1600

grrcn_tran_final['monto_sobrerango_v1'] =  grrcn_tran_final['sobrerango_v1']*100
grrcn_tran_final['iva_monto_sobrerango_v1'] =  grrcn_tran_final['monto_sobrerango_v1']*0.1600

grrcn_tran_final['monto_sobrerango_v2'] =  grrcn_tran_final['sobrerango_v2']*100
grrcn_tran_final['iva_monto_sobrerango_v2'] =  grrcn_tran_final['monto_sobrerango_v2']*0.1600

grrcn_tran_final['monto_sobrerango_v3'] =  grrcn_tran_final['sobrerango_v3']*100
grrcn_tran_final['iva_monto_sobrerango_v3'] =  grrcn_tran_final['monto_sobrerango_v3']*0.1600

grrcn_tran_final['monto_comision_industria_fee'] =  grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['industria_fee'])
grrcn_tran_final['iva_monto_comision_industria_fee'] =  grrcn_tran_final['monto_comision_industria_fee']*0.1600

grrcn_tran_final['monto_comision_inbound_fee'] = grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['inbound_fee'])
grrcn_tran_final['iva_monto_comision_inbound_fee'] =  grrcn_tran_final['monto_comision_inbound_fee']*0.1600

grrcn_tran_final['monto_comision_network_fee'] = grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['network_fee'])
grrcn_tran_final['iva_monto_comision_network_fee'] =  grrcn_tran_final['monto_comision_network_fee']*0.1600

grrcn_tran_final.loc[(grrcn_tran_final['monto_comision_network_fee'] < 0), 'monto_comision_cancelacion_network_fee'] = grrcn_tran_final['monto_comision_network_fee']*-1
grrcn_tran_final['iva_monto_comision_cancelacion_network_fee'] =  grrcn_tran_final['monto_comision_cancelacion_network_fee']*0.1600

grrcn_tran_final.loc[(grrcn_tran_final['monto_comision_inbound_fee'] < 0), 'monto_comision_cancelacion_inbound_fee'] = grrcn_tran_final['monto_comision_inbound_fee']*-1
grrcn_tran_final['iva_monto_comision_cancelacion_inbound_fee'] =  grrcn_tran_final['monto_comision_cancelacion_inbound_fee']*0.1600

grrcn_tran_final['monto_comision_cancelacion_network_fee'].fillna(0,inplace=True)
grrcn_tran_final['iva_monto_comision_cancelacion_network_fee'].fillna(0,inplace=True)
grrcn_tran_final['monto_comision_cancelacion_inbound_fee'].fillna(0,inplace=True)
grrcn_tran_final['iva_monto_comision_cancelacion_inbound_fee'].fillna(0,inplace=True)

grrcn_tran_final['monto_comision_cancelacion'] = (grrcn_tran_final['monto_comision_cancelacion_network_fee'] +
                                                  grrcn_tran_final['monto_comision_cancelacion_inbound_fee'])
grrcn_tran_final['iva_monto_comision_cancelacion'] = (grrcn_tran_final['iva_monto_comision_cancelacion_network_fee'] +
                                                      grrcn_tran_final['iva_monto_comision_cancelacion_inbound_fee'])

grrcn_tran_final['monto_comision_cancelacion'].fillna(0,inplace=True)
grrcn_tran_final['iva_monto_comision_cancelacion'].fillna(0,inplace=True)

grrcn_tran_final['monto_sobretasa'] = grrcn_tran_final['Monto_transaccion']*(grrcn_tran_final['sobretasa'])
grrcn_tran_final['iva_monto_sobretasa'] =  grrcn_tran_final['monto_sobretasa']*0.1600

grrcn_tran_final['total_monto_sobrerango'] =  (grrcn_tran_final['monto_sobrerango_v0'] +
                                               grrcn_tran_final['monto_sobrerango_v1'] +
                                               grrcn_tran_final['monto_sobrerango_v2'] +
                                               grrcn_tran_final['monto_sobrerango_v3'])
grrcn_tran_final['total_iva_monto_sobrerango'] =  (grrcn_tran_final['iva_monto_sobrerango_v0'] +
                                                   grrcn_tran_final['iva_monto_sobrerango_v1'] +
                                                   grrcn_tran_final['iva_monto_sobrerango_v2'] +
                                                   grrcn_tran_final['iva_monto_sobrerango_v3'])

grrcn_tran_final['total_monto_comisiones'] =  (grrcn_tran_final['monto_comision_industria_fee'] +
                                               grrcn_tran_final['monto_comision_inbound_fee'] +
                                               grrcn_tran_final['monto_comision_network_fee'] +
                                               grrcn_tran_final['monto_comision_cancelacion'])
grrcn_tran_final['total_iva_monto_comisiones'] =  (grrcn_tran_final['iva_monto_comision_industria_fee'] +
                                                    grrcn_tran_final['iva_monto_comision_inbound_fee'] +
                                                    grrcn_tran_final['iva_monto_comision_network_fee'] +
                                                    grrcn_tran_final['iva_monto_comision_cancelacion'])

grrcn_tran_final.head()

,1,2,3,4,fecha_pago,6,numero_establecimiento,fecha_presentacion,fecha_procesamiento,10,...,monto_comision_cancelacion_inbound_fee,iva_monto_comision_cancelacion_inbound_fee,monto_comision_cancelacion,iva_monto_comision_cancelacion,monto_sobretasa,iva_monto_sobretasa,total_monto_sobrerango,total_iva_monto_sobrerango,total_monto_comisiones,total_iva_monto_comisiones
0,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130336,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.01
1,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130337,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.01
2,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130338,...,0.00,0.00,0.00,0.00,108.00,17.28,0.40,0.06,46.44,7.43
3,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130339,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.19,0.67
4,TRANSACTN,8138352885,002,3350052821,2021-12-02,MXN,8138464136,2021-12-01,2021-12-01,000000000130340,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.84,1.73


In [63]:
grrcn_tran_final.shape

(1148339, 89)

In [64]:
grrcn_tran_range = grrcn_tran_final[(grrcn_tran_final['fecha_pago'] >= start_date) & (grrcn_tran_final['fecha_pago'] <= final_date)]

In [65]:
grrcn_tran_range.shape

(100762, 89)

In [66]:
# grrcn_tran.to_excel('DETALLE OPTBLUE TRANSACTION.xlsx',index=False)

# CONCILIACION ALL

In [85]:
cruce_1_previo = db.merge(grrcn_tran_final[['order_id','fecha_transaccion','fecha_pago','rechazo_amex','bin_bank',
                                            'last4_bank','Monto_transaccion','monto_sobrerango_v0','monto_sobrerango_v1',
                                                                      'monto_sobrerango_v2',
                                                                      'monto_sobrerango_v3',
                                                                      'monto_comision_network_fee',
                                                                      'monto_comision_inbound_fee',
                                                                      'monto_comision_industria_fee',
                                                                      'monto_comision_cancelacion',
                                                                      'iva_monto_sobrerango_v0',
                                                                      'iva_monto_sobrerango_v1',
                                                                      'iva_monto_sobrerango_v2',
                                                                      'iva_monto_sobrerango_v3',
                                                                      'iva_monto_comision_network_fee',
                                                                      'iva_monto_comision_inbound_fee',
                                                                      'iva_monto_comision_industria_fee',
                                                                      'iva_monto_comision_cancelacion',
                                                                      'total_monto_sobrerango',
                                                                      'total_monto_comisiones',
                                                                      'monto_sobretasa',
                                                                      'total_iva_monto_sobrerango',
                                                                      'total_iva_monto_comisiones',
                                                                      'iva_monto_sobretasa']],
                                        on='order_id',how='inner')
cruce_1_previo['Monto_transaccion'].fillna(0,inplace=True)
cruce_1_previo['monto_sobrerango_v0'].fillna(0,inplace=True)
cruce_1_previo['monto_sobrerango_v1'].fillna(0,inplace=True)
cruce_1_previo['monto_sobrerango_v2'].fillna(0,inplace=True)
cruce_1_previo['monto_sobrerango_v3'].fillna(0,inplace=True)
cruce_1_previo['monto_comision_network_fee'].fillna(0,inplace=True)
cruce_1_previo['monto_comision_inbound_fee'].fillna(0,inplace=True)
cruce_1_previo['monto_comision_cancelacion'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_sobrerango_v0'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_sobrerango_v1'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_sobrerango_v2'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_sobrerango_v3'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_comision_network_fee'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_comision_inbound_fee'].fillna(0,inplace=True)  
cruce_1_previo['monto_comision_industria_fee'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_comision_cancelacion'].fillna(0,inplace=True)
cruce_1_previo['total_monto_sobrerango'].fillna(0,inplace=True)
cruce_1_previo['total_monto_comisiones'].fillna(0,inplace=True)
cruce_1_previo['monto_sobretasa'].fillna(0,inplace=True)
cruce_1_previo['total_iva_monto_sobrerango'].fillna(0,inplace=True)
cruce_1_previo['total_iva_monto_comisiones'].fillna(0,inplace=True)
cruce_1_previo['iva_monto_sobretasa'].fillna(0,inplace=True)
cruce_1_previo['bin_bank_y'].fillna('NO ENCONTRADO',inplace=True)
cruce_1_previo['last4_bank'].fillna('NO ENCONTRADO',inplace=True)
cruce_1_previo['fecha_pago'].fillna('1900-01-01',inplace=True)
cruce_1_previo['fecha_pago'] = pd.to_datetime(cruce_1_previo['fecha_pago'])
cruce_1_previo['comparison'] = cruce_1_previo['Monto_transaccion'] == cruce_1_previo['amount_new']
cruce_1_previo.loc[((cruce_1_previo['status_code'] == '3') & (cruce_1_previo['Monto_transaccion'] > 0)),
                   "monto_sobretasa"] = 0
cruce_1_previo.loc[((cruce_1_previo['status_code'] == '3') & (cruce_1_previo['Monto_transaccion'] > 0)),
                   "iva_monto_sobretasa"] = 0
cruce_1_previo.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,0.48,1.56,0.00,0.00,14.25,0.00,0.00,2.28,0.00,True
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,0.00,0.24,0.00,0.00,1.70,0.00,0.00,0.27,0.00,True
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,0.00,0.32,0.00,0.00,2.29,0.00,0.00,0.37,0.00,True
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,0.00,0.51,0.00,0.00,3.55,0.00,0.00,0.57,0.00,True
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,0.00,0.62,0.00,0.00,4.50,0.00,0.00,0.72,0.00,True


In [86]:
cruce_1_previo.shape

(158984, 58)

In [87]:
cruce_1_previo['status_code'].unique()

array(['1', '-6', '3', '-5', '5'], dtype=object)

### CRUCE MONTOS TRANSACCION POSITIVOS

In [88]:
cruce_1_distinct_3 = cruce_1_previo[(cruce_1_previo['Monto_transaccion'] >= 0)]
cruce_1_distinct_3.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,0.48,1.56,0.00,0.00,14.25,0.00,0.00,2.28,0.00,True
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,0.00,0.24,0.00,0.00,1.70,0.00,0.00,0.27,0.00,True
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,0.00,0.32,0.00,0.00,2.29,0.00,0.00,0.37,0.00,True
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,0.00,0.51,0.00,0.00,3.55,0.00,0.00,0.57,0.00,True
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,0.00,0.62,0.00,0.00,4.50,0.00,0.00,0.72,0.00,True


In [89]:
cruce_1_distinct_3.shape

(158962, 58)

In [90]:
duplicates_1 = cruce_1_distinct_3[cruce_1_distinct_3.duplicated(['order_id'] , keep=False)]
# duplicates_1.sort_values(by=['order_id','fecha_transaccion'],inplace=True)
duplicates_1

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison


### CRUCE MONTOS TRANSACCION NEGATIVOS

In [91]:
cruce_1_same_3 = cruce_1_previo[(cruce_1_previo['Monto_transaccion'] < 0)]
cruce_1_same_3.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
8305,8dbf5480-bb1b-4d9b-8814-115c010a0e75,a7261e93-720e-4116-a29b-f53cac0e7e61,JSBY5FP0IK8L,001450638969481,8138464227,OPTBLUE,466219,5045,2021-12-21,2021-12-16 21:53:22.000,...,-19.20,-72.00,28.80,-1.30,-450.00,-0.00,-0.21,-72.00,-0.00,True
27657,6cf4ba3d-9464-4ce3-9593-bac159278f11,c8d0736d-ceb5-489e-9605-7ec57f158cb4,3BIFDJK5VKHA,001456281463482,8138464144,OPTBLUE,1317521,7399,2021-12-24,2021-12-24 05:04:31.000,...,-0.00,-31.67,3.42,-1.30,-197.95,-0.00,-0.21,-31.67,-0.00,True
30371,f45b560d-dfdc-45f3-a117-220ed7656570,a27ac94f-3c73-4ba2-bc1f-b91495ef86e4,XROMK8V4UBQE,001454520043488,8138464227,OPTBLUE,1147724,7699,2021-12-25,2021-12-22 01:51:23.000,...,-0.00,-72.00,9.60,-1.30,-450.00,-0.00,-0.21,-72.00,-0.00,True
30414,a285da19-b46c-40c9-ae49-062b65dd5683,2d8210e4-fbdd-4c47-86ed-f402d9f9b356,O9PKY1PYZBLC,001456362032487,8138464128,OPTBLUE,983188,4121,2021-12-25,2021-12-24 14:09:50.000,...,-9.60,-31.20,14.40,-1.30,-195.00,-0.00,-0.21,-31.20,-0.00,True
44983,5f01ba2d-0e91-4455-9fea-d5d8085b8dc6,ba9b464c-92d9-41c9-b4db-7ac687162a58,DLDENHZA3PSZ,001457245487484,8138464227,OPTBLUE,375291,7622,2021-12-28,2021-12-26 18:18:26.000,...,-0.00,-24.00,3.20,0.00,-150.00,-0.00,0.00,-24.00,-0.00,True


In [92]:
cruce_1_same_3.shape

(22, 58)

In [93]:
duplicates_1 = cruce_1_same_3[cruce_1_same_3.duplicated(['order_id'] , keep=False)]
# duplicates_1.sort_values(by=['order_id','fecha_transaccion'],inplace=True)
duplicates_1

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison


In [94]:
duplicates_1.shape

(0, 58)

### IDENTIFICACION DE CANCELACIONES

In [95]:
cruce_2_ident = cruce_1_distinct_3.merge(cruce_1_same_3[['order_id','affiliation_type']],on='order_id',how='left')
cruce_2_ident['affiliation_type_y'].fillna('sin_cancelacion',inplace=True)
cruce_2_ident.drop(cruce_2_ident[cruce_2_ident['affiliation_type_y'] != 'sin_cancelacion'].index, inplace = True)
cruce_2_ident.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type_x,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison,affiliation_type_y
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,1.56,0.00,0.00,14.25,0.00,0.00,2.28,0.00,True,sin_cancelacion
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,0.24,0.00,0.00,1.70,0.00,0.00,0.27,0.00,True,sin_cancelacion
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,0.32,0.00,0.00,2.29,0.00,0.00,0.37,0.00,True,sin_cancelacion
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,0.51,0.00,0.00,3.55,0.00,0.00,0.57,0.00,True,sin_cancelacion
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,0.62,0.00,0.00,4.50,0.00,0.00,0.72,0.00,True,sin_cancelacion


In [96]:
cruce_con_canc = pd.concat([cruce_2_ident,cruce_1_same_3])
cruce_con_canc.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type_x,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison,affiliation_type_y,affiliation_type
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,0.00,0.00,14.25,0.00,0.00,2.28,0.00,True,sin_cancelacion,NaN
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,0.00,0.00,1.70,0.00,0.00,0.27,0.00,True,sin_cancelacion,NaN
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,0.00,0.00,2.29,0.00,0.00,0.37,0.00,True,sin_cancelacion,NaN
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,0.00,0.00,3.55,0.00,0.00,0.57,0.00,True,sin_cancelacion,NaN
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,0.00,0.00,4.50,0.00,0.00,0.72,0.00,True,sin_cancelacion,NaN


In [97]:
cruce_con_canc.shape

(158962, 60)

### FILTRAR LOS RECHAZADOS AMEX

In [98]:
cruce_1 = cruce_1_previo[(cruce_1_previo['rechazo_amex'] != 'REJ')]
cruce_1.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,0.48,1.56,0.00,0.00,14.25,0.00,0.00,2.28,0.00,True
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,0.00,0.24,0.00,0.00,1.70,0.00,0.00,0.27,0.00,True
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,0.00,0.32,0.00,0.00,2.29,0.00,0.00,0.37,0.00,True
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,0.00,0.51,0.00,0.00,3.55,0.00,0.00,0.57,0.00,True
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,0.00,0.62,0.00,0.00,4.50,0.00,0.00,0.72,0.00,True


In [99]:
cruce_1.shape

(158969, 58)

### RECHAZADOS AMEX

In [100]:
cruce_1_rej = cruce_1_previo[(cruce_1_previo['rechazo_amex'] == 'REJ')]
cruce_1_rej.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
9825,3cfa55e8-f29e-4dff-8729-ea3722b967a4,3f681dfa-d8a3-4428-8b05-a12c6e21c146,CDVGGWR7XLYK,001454489330488,8138464144,OPTBLUE,91909,2842,2021-12-21,2021-12-22 01:06:23.000,...,0.67,3.10,0.00,0.40,25.65,0.00,0.06,4.10,0.00,True
29617,3cfa55e8-f29e-4dff-8729-ea3722b967a4,f96ebf6b-1c2d-44df-a048-cdc2caa6041c,Z8VX1QFSENNN,001457000963489,8138464144,OPTBLUE,91909,2842,2021-12-25,2021-12-25 20:44:35.000,...,0.87,4.04,0.00,0.40,33.47,0.00,0.06,5.35,0.00,True
42567,3cfa55e8-f29e-4dff-8729-ea3722b967a4,7795ce60-deb8-47e9-8788-42caf9dfd786,SDYGXO8OZIW5,001458503421488,8138464144,OPTBLUE,91909,2842,2021-12-28,2021-12-28 20:02:36.000,...,1.43,6.63,0.00,0.40,54.88,0.00,0.06,8.78,0.00,True
45209,3cfa55e8-f29e-4dff-8729-ea3722b967a4,b222574b-21dd-4fa4-ab0b-c74941d33ad8,IJDWVNPBLGNI,001458750255480,8138464144,OPTBLUE,91909,2842,2021-12-28,2021-12-29 01:22:40.000,...,0.88,4.07,0.00,0.40,33.71,0.00,0.06,5.39,0.00,True
58003,3cfa55e8-f29e-4dff-8729-ea3722b967a4,7c924255-4c51-4bd6-a8c5-902c5b470d18,YRCOQNGDIYY5,001460026264486,8138464144,OPTBLUE,91909,2842,2021-12-30,2021-12-31 01:29:52.000,...,1.18,5.45,0.00,0.40,45.08,0.00,0.06,7.21,0.00,True


In [101]:
cruce_1_rej.shape

(15, 58)

In [102]:
cruce_1_rej.to_csv('rej.csv')

# PAYMENTS CRUCE GRRCN VS CLIP

In [120]:
txn_grrcn_deposit = cruce_1
# txn_grrcn_deposit = cruce_1[(cruce_1['fecha_pago'] >= start_date) & (cruce_1['fecha_pago'] <= final_date)] 
txn_grrcn_deposit.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
0,d671ee4e-40b8-4ede-a105-304f044be0a0,24208d60-f269-4228-acea-f9250a21d152,JSLUWR8TVTEQ,001453025292483,8138464128,OPTBLUE,702730,4121,2021-12-20,2021-12-20 06:00:24.000,...,0.48,1.56,0.00,0.00,14.25,0.00,0.00,2.28,0.00,True
1,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,e94240b1-b28b-4145-a2d3-c962cd5b45d2,AAGQHWJVP62L,001453018851485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:00:28.000,...,0.00,0.24,0.00,0.00,1.70,0.00,0.00,0.27,0.00,True
2,46c1ddea-dc26-422d-936a-4d89bcbb1ff5,2fcd6e4f-6b0e-468e-9c7c-a4f457730c8a,HTQH36XWKT1T,001453023433485,8138464227,OPTBLUE,332456,7210,2021-12-20,2021-12-20 06:02:29.000,...,0.00,0.32,0.00,0.00,2.29,0.00,0.00,0.37,0.00,True
3,05b27885-1e84-4707-becf-08430c198280,84828102-6fbe-4657-8b10-fe581e8e0402,GZ1OVVUY5H5G,001453020571485,8138464136,OPTBLUE,20558,5812,2021-12-20,2021-12-20 06:03:01.000,...,0.00,0.51,0.00,0.00,3.55,0.00,0.00,0.57,0.00,True
4,f936610b-62ac-44cf-9bae-69bf29434b3e,02f6ab9a-16b5-45b7-ac79-0b1bea4ee168,MJYCGQD0QM1C,001453018868489,8138464128,OPTBLUE,1279672,4121,2021-12-20,2021-12-20 06:03:40.000,...,0.00,0.62,0.00,0.00,4.50,0.00,0.00,0.72,0.00,True


In [121]:
txn_grrcn_deposit.shape

(158969, 58)

In [123]:
txn_grrcn_deposit.to_csv('check_1.csv')

In [124]:
txn_grrcn_deposit['fecha_pago'].unique()

array(['2021-12-21T00:00:00.000000000', '2021-12-22T00:00:00.000000000',
       '2021-12-17T00:00:00.000000000', '2021-12-03T00:00:00.000000000',
       '2021-12-23T00:00:00.000000000', '2021-12-24T00:00:00.000000000',
       '2021-12-27T00:00:00.000000000', '2021-12-28T00:00:00.000000000',
       '2021-12-29T00:00:00.000000000', '2021-12-30T00:00:00.000000000',
       '2021-12-31T00:00:00.000000000', '2022-01-03T00:00:00.000000000',
       '2022-01-04T00:00:00.000000000', '2022-01-05T00:00:00.000000000',
       '2022-01-06T00:00:00.000000000', '2022-01-07T00:00:00.000000000',
       '2022-01-10T00:00:00.000000000', '2022-01-11T00:00:00.000000000',
       '2022-01-12T00:00:00.000000000', '2022-01-13T00:00:00.000000000',
       '2022-01-14T00:00:00.000000000', '2022-01-17T00:00:00.000000000',
       '2022-01-18T00:00:00.000000000', '2022-01-19T00:00:00.000000000',
       '2022-01-20T00:00:00.000000000'], dtype='datetime64[ns]')

In [125]:
txn_grrcn_deposit.columns

Index(['merchant_id', 'transaction_id', 'order_id', 'clave_autorizacion',
       'affiliation', 'affiliation_type', 'sub_affiliation', 'mcc',
       'date_transaction', 'created_at', 'updated_at', 'psp', 'amount',
       'bin_bank_x', 'receipt_no', 'last4', 'status_code', 'decline_code',
       'msi', 'device_id', 'clip_reader', 'status_msg', 'issuer', 'brand',
       'amount_new', 'INDEX_TOTAL', 'INDEX_PARCIAL', 'index_comisiones',
       'index_comisiones_industria', 'fecha_transaccion', 'fecha_pago',
       'rechazo_amex', 'bin_bank_y', 'last4_bank', 'Monto_transaccion',
       'monto_sobrerango_v0', 'monto_sobrerango_v1', 'monto_sobrerango_v2',
       'monto_sobrerango_v3', 'monto_comision_network_fee',
       'monto_comision_inbound_fee', 'monto_comision_industria_fee',
       'monto_comision_cancelacion', 'iva_monto_sobrerango_v0',
       'iva_monto_sobrerango_v1', 'iva_monto_sobrerango_v2',
       'iva_monto_sobrerango_v3', 'iva_monto_comision_network_fee',
       'iva_monto_com

In [126]:
duplicates_1 = txn_grrcn_deposit[txn_grrcn_deposit.duplicated(['order_id'] , keep=False)]
# duplicates_1.sort_values(by=['order_id','fecha_transaccion'],inplace=True)
duplicates_1

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
8304,8dbf5480-bb1b-4d9b-8814-115c010a0e75,a7261e93-720e-4116-a29b-f53cac0e7e61,JSBY5FP0IK8L,001450638969481,8138464227,OPTBLUE,466219,5045,2021-12-21,2021-12-16 21:53:22.000,...,19.20,72.00,0.00,1.30,630.00,0.00,0.21,100.80,0.00,False
8305,8dbf5480-bb1b-4d9b-8814-115c010a0e75,a7261e93-720e-4116-a29b-f53cac0e7e61,JSBY5FP0IK8L,001450638969481,8138464227,OPTBLUE,466219,5045,2021-12-21,2021-12-16 21:53:22.000,...,-19.20,-72.00,28.80,-1.30,-450.00,-0.00,-0.21,-72.00,-0.00,True
27656,6cf4ba3d-9464-4ce3-9593-bac159278f11,c8d0736d-ceb5-489e-9605-7ec57f158cb4,3BIFDJK5VKHA,001456281463482,8138464144,OPTBLUE,1317521,7399,2021-12-24,2021-12-24 05:04:31.000,...,0.00,31.67,0.00,1.30,219.35,0.00,0.21,35.10,0.00,False
27657,6cf4ba3d-9464-4ce3-9593-bac159278f11,c8d0736d-ceb5-489e-9605-7ec57f158cb4,3BIFDJK5VKHA,001456281463482,8138464144,OPTBLUE,1317521,7399,2021-12-24,2021-12-24 05:04:31.000,...,-0.00,-31.67,3.42,-1.30,-197.95,-0.00,-0.21,-31.67,-0.00,True
30370,f45b560d-dfdc-45f3-a117-220ed7656570,a27ac94f-3c73-4ba2-bc1f-b91495ef86e4,XROMK8V4UBQE,001454520043488,8138464227,OPTBLUE,1147724,7699,2021-12-25,2021-12-22 01:51:23.000,...,0.00,72.00,0.00,1.30,510.00,0.00,0.21,81.60,0.00,False
30371,f45b560d-dfdc-45f3-a117-220ed7656570,a27ac94f-3c73-4ba2-bc1f-b91495ef86e4,XROMK8V4UBQE,001454520043488,8138464227,OPTBLUE,1147724,7699,2021-12-25,2021-12-22 01:51:23.000,...,-0.00,-72.00,9.60,-1.30,-450.00,-0.00,-0.21,-72.00,-0.00,True
30414,a285da19-b46c-40c9-ae49-062b65dd5683,2d8210e4-fbdd-4c47-86ed-f402d9f9b356,O9PKY1PYZBLC,001456362032487,8138464128,OPTBLUE,983188,4121,2021-12-25,2021-12-24 14:09:50.000,...,-9.60,-31.20,14.40,-1.30,-195.00,-0.00,-0.21,-31.20,-0.00,True
30415,a285da19-b46c-40c9-ae49-062b65dd5683,2d8210e4-fbdd-4c47-86ed-f402d9f9b356,O9PKY1PYZBLC,001456362032487,8138464128,OPTBLUE,983188,4121,2021-12-25,2021-12-24 14:09:50.000,...,9.60,31.20,0.00,1.30,285.00,0.00,0.21,45.60,0.00,False
44982,5f01ba2d-0e91-4455-9fea-d5d8085b8dc6,ba9b464c-92d9-41c9-b4db-7ac687162a58,DLDENHZA3PSZ,001457245487484,8138464227,OPTBLUE,375291,7622,2021-12-28,2021-12-26 18:18:26.000,...,0.00,24.00,0.00,1.30,170.00,0.00,0.21,27.20,0.00,False
44983,5f01ba2d-0e91-4455-9fea-d5d8085b8dc6,ba9b464c-92d9-41c9-b4db-7ac687162a58,DLDENHZA3PSZ,001457245487484,8138464227,OPTBLUE,375291,7622,2021-12-28,2021-12-26 18:18:26.000,...,-0.00,-24.00,3.20,0.00,-150.00,-0.00,0.00,-24.00,-0.00,True


In [127]:
duplicates_1.shape

(44, 58)

In [128]:
duplicates_1['status_code'].unique()

array(['3'], dtype=object)

### COMPARISON

In [129]:
comparison_1 = txn_grrcn_deposit[(txn_grrcn_deposit['comparison'] == False)]
comparison_1.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison
8304,8dbf5480-bb1b-4d9b-8814-115c010a0e75,a7261e93-720e-4116-a29b-f53cac0e7e61,JSBY5FP0IK8L,001450638969481,8138464227,OPTBLUE,466219,5045,2021-12-21,2021-12-16 21:53:22.000,...,19.20,72.00,0.00,1.30,630.00,0.00,0.21,100.80,0.00,False
27656,6cf4ba3d-9464-4ce3-9593-bac159278f11,c8d0736d-ceb5-489e-9605-7ec57f158cb4,3BIFDJK5VKHA,001456281463482,8138464144,OPTBLUE,1317521,7399,2021-12-24,2021-12-24 05:04:31.000,...,0.00,31.67,0.00,1.30,219.35,0.00,0.21,35.10,0.00,False
30370,f45b560d-dfdc-45f3-a117-220ed7656570,a27ac94f-3c73-4ba2-bc1f-b91495ef86e4,XROMK8V4UBQE,001454520043488,8138464227,OPTBLUE,1147724,7699,2021-12-25,2021-12-22 01:51:23.000,...,0.00,72.00,0.00,1.30,510.00,0.00,0.21,81.60,0.00,False
30415,a285da19-b46c-40c9-ae49-062b65dd5683,2d8210e4-fbdd-4c47-86ed-f402d9f9b356,O9PKY1PYZBLC,001456362032487,8138464128,OPTBLUE,983188,4121,2021-12-25,2021-12-24 14:09:50.000,...,9.60,31.20,0.00,1.30,285.00,0.00,0.21,45.60,0.00,False
44982,5f01ba2d-0e91-4455-9fea-d5d8085b8dc6,ba9b464c-92d9-41c9-b4db-7ac687162a58,DLDENHZA3PSZ,001457245487484,8138464227,OPTBLUE,375291,7622,2021-12-28,2021-12-26 18:18:26.000,...,0.00,24.00,0.00,1.30,170.00,0.00,0.21,27.20,0.00,False


In [130]:
comparison_1.shape

(22, 58)

### NO ENCONTRADOS

In [131]:
not_found_txn_epa_deposit = txn_grrcn_deposit[(txn_grrcn_deposit['last4_bank'] == 'NO ENCONTRADO')]
not_found_txn_epa_deposit.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa,comparison


In [132]:
not_found_txn_epa_deposit.shape

(0, 58)

# GROUP BY DEPOSIT DATE AND STATUS

### POR FECHA DE DEPOSITO

In [133]:
deposit_date_cruce_1 = txn_grrcn_deposit.groupby(['fecha_pago']).agg({'transaction_id':'count',
                                                                      'amount':sum,
                                                                      'amount_new':sum,
                                                                      'Monto_transaccion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_date_cruce_1.head()

,fecha_pago,transaction_id,amount,amount_new,Monto_transaccion,monto_sobrerango_v0,monto_sobrerango_v1,monto_sobrerango_v2,monto_sobrerango_v3,monto_comision_network_fee,...,iva_monto_comision_network_fee,iva_monto_comision_inbound_fee,iva_monto_comision_industria_fee,iva_monto_comision_cancelacion,total_monto_sobrerango,total_monto_comisiones,monto_sobretasa,total_iva_monto_sobrerango,total_iva_monto_comisiones,iva_monto_sobretasa
0,2021-12-03,1,"8,500.00","8,500.00","8,500.00",0.00,0.00,0.70,0.00,17.00,...,2.72,0.00,5.44,0.00,0.70,51.00,0.00,0.11,8.16,0.00
1,2021-12-17,1,"30,000.00","-30,000.00","30,000.00",0.00,0.00,0.00,1.30,60.00,...,9.60,19.20,72.00,0.00,1.30,630.00,0.00,0.21,100.80,0.00
2,2021-12-21,4595,"13,805,187.47","13,805,187.47","13,805,187.47",0.00,457.60,386.40,395.20,"27,610.37",...,"4,417.66","1,997.47","33,705.81",0.00,"1,239.20","250,755.91","102,236.43",198.27,"40,120.95","16,357.83"
3,2021-12-22,5597,"14,435,829.42","14,315,829.42","14,375,829.42",0.00,564.80,448.70,380.90,"28,751.66",...,"4,600.27","2,255.31","35,825.26",28.80,"1,394.40","266,935.26","64,432.01",223.10,"42,709.64","10,309.12"
4,2021-12-23,6266,"16,642,333.57","16,642,333.57","16,642,333.57",0.00,644.00,556.50,426.40,"33,284.67",...,"5,325.55","2,528.59","41,009.68",0.00,"1,626.90","305,398.87","62,420.89",260.30,"48,863.82","9,987.34"


In [134]:
#industria_fee	network_fee	sobretasa	inbound_fee	sobrerango_v2	sobrerango_v3	sobrerango_v1	sobrerango_v0

In [135]:
deposit_date_cruce_1.shape

(25, 27)

In [137]:
deposit_date_cruce_1.to_csv('deposito_check.csv')

### CRUCE POR MESES SIN INTERESES

In [ ]:
deposit_date_msi = txn_grrcn_deposit.groupby(['numero_cuota','fecha_pago']).agg({'transaction_id':'count',
                                                                                 'Monto_transaccion':sum,
                                                                                 'amount':sum,
                                                                                 'monto_sobretasa':sum,
                                                                                 'iva_monto_sobretasa':sum}).reset_index()
deposit_date_msi.head()

In [ ]:
deposit_date_msi.shape

### CRUCE CON CONTRACARGOS

In [ ]:
deposit_date_cruce_2 = deposit_date_cruce_1.merge(deposit_date_chbk[['fecha_pago','Monto_bruto_contracargo'
                                                                     ,'Monto_descuento_contracargo','Monto_servicio_contracargo'
                                                                     ,'Monto_impuesto_contracargo','Monto_neto_contracargo']]
                                                  ,on='fecha_pago',how='left')
deposit_date_cruce_2['Monto_bruto_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_descuento_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_servicio_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_impuesto_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2['Monto_neto_contracargo'].fillna(0,inplace=True)
deposit_date_cruce_2.head()

In [ ]:
deposit_date_cruce_2.shape

### CRUCE CON AJUSTES

In [ ]:
deposit_date_cruce_3 = deposit_date_cruce_2.merge(deposit_date_adj[['fecha_pago',
                                                                     'Monto_bruto_adjustment',
                                                                     'Monto_descuento_adjustment',
                                                                    'Monto_servicio_adjustment',
                                                                    'Monto_impuesto_adjustment',
                                                                    'Monto_neto_adjustment']],
                                                  on='fecha_pago',how='left')
deposit_date_cruce_3['Monto_bruto_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_descuento_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_servicio_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_impuesto_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_neto_adjustment'].fillna(0,inplace=True)
deposit_date_cruce_3['Monto_ajustes'] = (deposit_date_cruce_3['Monto_neto_adjustment'] +
                                             deposit_date_cruce_3['Monto_neto_contracargo'])
deposit_date_cruce_3['Monto_ajustes_calc'] = (-deposit_date_cruce_3['monto_sobretasa'] +
                                              -deposit_date_cruce_3['iva_monto_sobretasa'] +
                                             deposit_date_cruce_3['Monto_neto_contracargo'])
deposit_date_cruce_3['Monto_impuesto_calc'] = (deposit_date_cruce_3['total_iva_monto_sobrerango'] +
                                              deposit_date_cruce_3['total_iva_monto_comisiones'])
deposit_date_cruce_3['Monto_neto_calc'] = (deposit_date_cruce_3['Monto_transaccion'] -
                                          deposit_date_cruce_3['total_monto_sobrerango'] -
                                          deposit_date_cruce_3['total_monto_comisiones'] -
                                          deposit_date_cruce_3['Monto_impuesto_calc'] +
                                          deposit_date_cruce_3['Monto_ajustes_calc'])
deposit_date_cruce_3.head()

In [ ]:
deposit_date_cruce_3.shape

### CRUCE FINAL DESGLOSADO

In [ ]:
deposit_date_cruce_final_desg = deposit_date_cruce_3[['fecha_pago','transaction_id','Monto_transaccion','monto_sobrerango_v0','monto_sobrerango_v1',
                                                 'monto_sobrerango_v2','monto_sobrerango_v3','monto_comision_network_fee',
                                                 'monto_comision_inbound_fee','monto_comision_industria_fee',
                                                 'monto_comision_cancelacion','iva_monto_sobrerango_v0',
                                                 'iva_monto_sobrerango_v1','iva_monto_sobrerango_v2','iva_monto_sobrerango_v3',
                                                 'iva_monto_comision_network_fee','iva_monto_comision_inbound_fee',
                                                 'iva_monto_comision_industria_fee','iva_monto_comision_cancelacion',
                                                 'total_monto_sobrerango','total_monto_comisiones','monto_sobretasa',
                                                 'total_iva_monto_sobrerango','total_iva_monto_comisiones','iva_monto_sobretasa',
                                                 'Monto_bruto_contracargo','Monto_descuento_contracargo',
                                                 'Monto_servicio_contracargo','Monto_impuesto_contracargo',
                                                 'Monto_neto_contracargo','Monto_bruto_adjustment','Monto_descuento_adjustment',
                                                 'Monto_servicio_adjustment','Monto_impuesto_adjustment','Monto_neto_adjustment',
                                                 'Monto_ajustes','Monto_ajustes_calc','Monto_impuesto_calc','Monto_neto_calc']]
deposit_date_cruce_final_desg.head()

In [ ]:
deposit_date_cruce_final_desg.shape

### CRUCE FINAL

In [ ]:
deposit_date_cruce_4 = grrcn_summ_range.merge(deposit_date_cruce_final_desg[['fecha_pago','total_monto_comisiones',
                                                                     'total_monto_sobrerango',
                                                                    'Monto_ajustes_calc',
                                                                    'Monto_impuesto_calc',
                                                                    'Monto_neto_calc']],
                                                  on='fecha_pago',how='left')
deposit_date_cruce_4.head()

In [ ]:
deposit_date_cruce_final = deposit_date_cruce_4[['fecha_pago','Monto_bruto','Monto_descuento','Monto_servicio','Monto_ajuste',
                                                 'Monto_impuestos','Monto_saldo_debito_inicial','Monto_neto',
                                                 'total_monto_comisiones','total_monto_sobrerango','Monto_ajustes_calc',
                                                 'Monto_impuesto_calc','Monto_neto_calc']]
deposit_date_cruce_final.head()

In [ ]:
deposit_date_cruce_final.shape

### TRANSACCIONES POSITIVAS

In [ ]:
deposit_success_txn = txn_grrcn_deposit[(txn_grrcn_deposit['Monto_transaccion'] >= 0)]

In [ ]:
deposit_success_cruce_1 = deposit_success_txn.groupby(['fecha_pago']).agg({'transaction_id':'count',
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_success_cruce_1.head()

In [ ]:
deposit_success_cruce_1.shape

### CANCELACIONES

In [ ]:
deposit_refunded_txn = txn_grrcn_deposit[(txn_grrcn_deposit['Monto_transaccion'] < 0)]

In [ ]:
deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['fecha_pago']).agg({'transaction_id':'count',
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_refunded_cruce_1.head()

In [ ]:
deposit_refunded_cruce_1.shape

### POR FECHA DE DEPOSITO Y AFILIACIÓN 

In [ ]:
deposit_afi_cruce_1 = txn_grrcn_deposit.groupby(['fecha_pago','numero_establecimiento','fecha_transaccion']).agg({
                                                                      'transaction_id':'count',                                                                  
                                                                      'Monto_transaccion':sum,
                                                                      'Monto_primera_cuota':sum,
                                                                      'Monto_cuota_siguiente':sum,
                                                                      'Monto_servicio':sum,
                                                                      'Monto_aceleracion':sum,
                                                                      'monto_sobrerango_v0':sum,
                                                                      'monto_sobrerango_v1':sum,
                                                                      'monto_sobrerango_v2':sum,
                                                                      'monto_sobrerango_v3':sum,
                                                                      'monto_comision_network_fee':sum,
                                                                      'monto_comision_inbound_fee':sum,
                                                                      'monto_comision_industria_fee':sum,
                                                                      'monto_comision_cancelacion':sum,
                                                                      'iva_monto_sobrerango_v0':sum,
                                                                      'iva_monto_sobrerango_v1':sum,
                                                                      'iva_monto_sobrerango_v2':sum,
                                                                      'iva_monto_sobrerango_v3':sum,
                                                                      'iva_monto_comision_network_fee':sum,
                                                                      'iva_monto_comision_inbound_fee':sum,
                                                                      'iva_monto_comision_industria_fee':sum,
                                                                      'iva_monto_comision_cancelacion':sum,
                                                                      'total_monto_sobrerango':sum,
                                                                      'total_monto_comisiones':sum,
                                                                      'monto_sobretasa':sum,
                                                                      'total_iva_monto_sobrerango':sum,
                                                                      'total_iva_monto_comisiones':sum,
                                                                      'iva_monto_sobretasa':sum}).reset_index()
deposit_afi_cruce_1.head()

In [ ]:
deposit_afi_cruce_1.shape

# TRANSACTIONS GRRNC - CLIP

In [ ]:
txn_grrcn_txn = cruce_1[(cruce_1['fecha_transaccion'] >= start_date) & (cruce_1['fecha_transaccion'] <= final_date)]
txn_grrcn_txn.sort_values(by=['fecha_transaccion','hora_transaccion'],inplace=True)
txn_grrcn_txn.head()

In [ ]:
txn_grrcn_txn.shape

In [ ]:
duplicates_2 = txn_grrcn_txn[txn_grrcn_txn.duplicated(['order_id'] , keep=False)]
duplicates_2.sort_values(by=['order_id','fecha_transaccion','hora_transaccion'],inplace=True)
duplicates_2.head()


In [ ]:
duplicates_2.shape

In [ ]:
txn_grrcn_txn.drop_duplicates(subset='order_id',keep='last',inplace=True)
txn_grrcn_txn.shape

In [ ]:
comparison_2 = txn_grrcn_txn[(txn_grrcn_txn['comparison'] == False)]
comparison_2.head()

In [ ]:
comparison_2.shape

# GROUP BY TRANSACTION DATE

In [ ]:
txn_txn_date_cruce_1 = txn_grrcn_txn.groupby(['fecha_transaccion']).agg({
    'transaction_id':'count',
    'Monto_transaccion':sum,
    'Monto_primera_cuota':sum,
    'Monto_cuota_siguiente':sum,
    'Monto_servicio':sum,
    'Monto_aceleracion':sum,
    'monto_sobrerango_v0':sum,
    'monto_sobrerango_v1':sum,
    'monto_sobrerango_v2':sum,
    'monto_sobrerango_v3':sum,
    'monto_comision_network_fee':sum,
    'monto_comision_inbound_fee':sum,
    'monto_comision_industria_fee':sum,
    'monto_comision_cancelacion':sum,
    'iva_monto_sobrerango_v0':sum,
    'iva_monto_sobrerango_v1':sum,
    'iva_monto_sobrerango_v2':sum,
    'iva_monto_sobrerango_v3':sum,
    'iva_monto_comision_network_fee':sum,
    'iva_monto_comision_inbound_fee':sum,
    'iva_monto_comision_industria_fee':sum,
    'iva_monto_comision_cancelacion':sum,
    'total_monto_sobrerango':sum,
    'total_monto_comisiones':sum,
    'monto_sobretasa':sum,
    'total_iva_monto_sobrerango':sum,
    'total_iva_monto_comisiones':sum,
    'iva_monto_sobretasa':sum,
    'amount_new':sum,
    'order_id':'count'}).reset_index()
txn_txn_date_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_date_cruce_1['diff'] = txn_txn_date_cruce_1['Monto_transaccion'] - txn_txn_date_cruce_1['amount_new']
txn_txn_date_cruce_1.head()


In [ ]:
txn_txn_date_cruce_1.shape

# GROUP BY TRANSACTION DATE AND STATUS CODE

In [ ]:
txn_txn_status_cruce_1 = txn_grrcn_txn.groupby(['fecha_transaccion','status_code','numero_establecimiento']).agg({
    'transaction_id':'count',
    'Monto_transaccion':sum,
    'Monto_primera_cuota':sum,
    'Monto_cuota_siguiente':sum,
    'Monto_servicio':sum,
    'Monto_aceleracion':sum,
    'monto_sobrerango_v0':sum,
    'monto_sobrerango_v1':sum,
    'monto_sobrerango_v2':sum,
    'monto_sobrerango_v3':sum,
    'monto_comision_network_fee':sum,
    'monto_comision_inbound_fee':sum,
    'monto_comision_industria_fee':sum,
    'monto_comision_cancelacion':sum,
    'iva_monto_sobrerango_v0':sum,
    'iva_monto_sobrerango_v1':sum,
    'iva_monto_sobrerango_v2':sum,
    'iva_monto_sobrerango_v3':sum,
    'iva_monto_comision_network_fee':sum,
    'iva_monto_comision_inbound_fee':sum,
    'iva_monto_comision_industria_fee':sum,
    'iva_monto_comision_cancelacion':sum,
    'total_monto_sobrerango':sum,
    'total_monto_comisiones':sum,
    'monto_sobretasa':sum,
    'total_iva_monto_sobrerango':sum,
    'total_iva_monto_comisiones':sum,
    'iva_monto_sobretasa':sum,
    'amount_new':sum,
    'order_id':'count'}).reset_index()
txn_txn_status_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_status_cruce_1['diff'] = txn_txn_status_cruce_1['Monto_transaccion'] - txn_txn_status_cruce_1['amount_new']
txn_txn_status_cruce_1.head()


In [ ]:
txn_txn_status_cruce_1.dtypes

In [ ]:
txn_txn_status_cruce_1.shape

### STATUS 1

In [ ]:
txn_txn_status_cruce_1_status_1 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '1')]
txn_txn_status_cruce_1_status_1.head()

In [ ]:
txn_txn_status_cruce_1_status_1.shape

### STATUS 3

In [ ]:
txn_txn_status_cruce_1_status_3 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '3')]
txn_txn_status_cruce_1_status_3.head()

In [ ]:
txn_txn_status_cruce_1_status_3.shape

# POSIBLES DISPUTAS

In [ ]:
posibles_disputas = txn_grrcn_txn[(txn_grrcn_txn['status_code'] != '1') & (txn_grrcn_txn['status_code'] != '3')]
posibles_disputas.head()


In [ ]:
posibles_disputas.shape

In [ ]:
txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['fecha_transaccion','numero_establecimiento']).agg({
    'transaction_id':'count',
    'Monto_transaccion':sum,
    'Monto_primera_cuota':sum,
    'Monto_cuota_siguiente':sum,
    'Monto_servicio':sum,
    'Monto_aceleracion':sum,
    'monto_sobrerango_v0':sum,
    'monto_sobrerango_v1':sum,
    'monto_sobrerango_v2':sum,
    'monto_sobrerango_v3':sum,
    'monto_comision_network_fee':sum,
    'monto_comision_inbound_fee':sum,
    'monto_comision_industria_fee':sum,
    'monto_comision_cancelacion':sum,
    'iva_monto_sobrerango_v0':sum,
    'iva_monto_sobrerango_v1':sum,
    'iva_monto_sobrerango_v2':sum,
    'iva_monto_sobrerango_v3':sum,
    'iva_monto_comision_network_fee':sum,
    'iva_monto_comision_inbound_fee':sum,
    'iva_monto_comision_industria_fee':sum,
    'iva_monto_comision_cancelacion':sum,
    'total_monto_sobrerango':sum,
    'total_monto_comisiones':sum,
    'monto_sobretasa':sum,
    'total_iva_monto_sobrerango':sum,
    'total_iva_monto_comisiones':sum,
    'iva_monto_sobretasa':sum,
    'amount_new':sum,
    'order_id':'count'}).reset_index()
txn_txn_status_cruce_1_posibles_disputas.head()

In [ ]:
txn_txn_status_cruce_1_posibles_disputas.shape

# EXCEL GRRCN 

In [ ]:
writer = pd.ExcelWriter('GRRCN ENERO 2022.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
# grrcn_file.to_excel(writer, sheet_name='GRRCN ALL',index=False)
grrcn_head.to_excel(writer, sheet_name='GRRCN HEADER',index=False)
grrcn_summ_range.to_excel(writer, sheet_name='GRRCN SUMMARY',index=False)
grrcn_subm_range.to_excel(writer, sheet_name='GRRCN SUBMISSION',index=False)
grrcn_tran_range.to_excel(writer, sheet_name='GRRCN TRANSACTION',index=False)
grrcn_txnp_range_1.to_excel(writer, sheet_name='GRRCN PRICING_1',index=False)
grrcn_txnp_range_2.to_excel(writer, sheet_name='GRRCN PRICING_2',index=False)
grrcn_char_range.to_excel(writer, sheet_name='GRRCN CHARGEBACKS',index=False)
grrcn_adju_range.to_excel(writer, sheet_name='GRRCN ADJUSTMENTS',index=False)
grrcn_trai.to_excel(writer, sheet_name='GRRCN TRAILER',index=False)
writer.save()
writer.close()

# PAYOUTS GRRNC

In [ ]:
writer = pd.ExcelWriter('OPTBLUE PAYOUTS ENERO 2022.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
deposit_date_cruce_final.to_excel(writer, sheet_name='PAYOUTS',index=False)
deposit_date_cruce_final_desg.to_excel(writer, sheet_name='PAYOUTS DESGLOSADO',index=False)
deposit_date_msi.to_excel(writer, sheet_name='PAYOUTS POR MSI',index=False)
deposit_success_cruce_1.to_excel(writer, sheet_name='PAYOUTS SUCCESS TXN',index=False)
deposit_refunded_cruce_1.to_excel(writer, sheet_name='PAYOUTS REFUNDED TXN',index=False)
deposit_afi_cruce_1.to_excel(writer, sheet_name='PAYOUTS AFILIACION',index=False)
txn_grrcn_deposit.to_excel(writer, sheet_name='PAYOUTS DETAIL',index=False)
cruce_1_rej.to_excel(writer, sheet_name='RECHAZADOS AMEX',index=False)
duplicates_1.to_excel(writer, sheet_name='PAYOUTS DUPLICATES',index=False)
comparison_1.to_excel(writer, sheet_name='PAYOUTS COMPARISON',index=False)
not_found_txn_epa_deposit.to_excel(writer, sheet_name='NO ENCONTRADOS',index=False)
grrcn_char.to_excel(writer, sheet_name='CHARGEBACKS DETAIL',index=False)
deposit_date_chbk.to_excel(writer, sheet_name='CHARGEBACKS BY DATE',index=False)
deposit_date_chnk_positivos.to_excel(writer, sheet_name='CHARGEBACKS POSITIVOS',index=False)
deposit_date_chnk_negativos.to_excel(writer, sheet_name='CHARGEBACKS NEGATIVOS',index=False)
grrcn_adju.to_excel(writer, sheet_name='ADJUSTMENTS DETAIL',index=False)
deposit_date_adj.to_excel(writer, sheet_name='ADJUSTMENTS BY DATE',index=False)
deposit_date_adj_positivos.to_excel(writer, sheet_name='ADJUSTMENTS POSITIVOS',index=False)
deposit_date_adj_negativos.to_excel(writer, sheet_name='ADJUSTMENTS NEGATIVOS',index=False)
writer.save()
writer.close()

# GRRCN TRANSACCIONES

In [ ]:
writer = pd.ExcelWriter('OPTBLUE TRANSACCIONES ENERO 2022.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
txn_grrcn_txn.to_excel(writer, sheet_name='TRANSACTIONS DETAIL',index=False)
duplicates_2.to_excel(writer, sheet_name='TRANSACTIONS DUPLICATES',index=False)
comparison_2.to_excel(writer, sheet_name='TRANSACTIONS COMPARISON',index=False)
txn_txn_date_cruce_1.to_excel(writer, sheet_name='TXN DATE',index=False)
txn_txn_status_cruce_1.to_excel(writer, sheet_name='TXN DATE AND STATUS',index=False)
txn_txn_status_cruce_1_status_1.to_excel(writer, sheet_name='STATUS 1',index=False)
txn_txn_status_cruce_1_status_3.to_excel(writer, sheet_name='STATUS 3',index=False)
txn_txn_status_cruce_1_posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS DATE-AFFI',index=False)
posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS',index=False)
writer.save()
writer.close()